In [8]:
import os
from tqdm.notebook import tqdm
from shutil import copyfile, copytree, move
import random

In [12]:
!wget https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7-tiny.pt

"wget" ­Ґ пў«пҐвбп ў­гваҐ­­Ґ© Ё«Ё ў­Ґи­Ґ©
Є®¬ ­¤®©, ЁбЇ®«­пҐ¬®© Їа®Ја ¬¬®© Ё«Ё Ї ЄҐв­л¬ д ©«®¬.


In [9]:
images = os.listdir('data/images/train')
labels = os.listdir('data/labels/train')
# s = []
random.shuffle(images)
# for image_name in tqdm(images):
#     l1 = image_name
#     l2 = '.'.join(l1.split('.')[:-1])
#     if os.path.exists(os.path.join('data/labels/train', str(l2)+'.txt')):
#         pass
#     else:
#         s.append(l1)
# print(s)

# l1 ='37_JPG.rf.a75cce59ed7182eee7aa119feb716230.jpg'
# l2 = '.'.join(l1.split('.')[:-1])

# move(os.path.join('data/images/train', l1), 
#          os.path.join('data/images/valid', l1))
# if os.path.exists(os.path.join('data/labels/train', str(l2)+'.txt')):
#         move(os.path.join('data/labels/train', str(l2)+'.txt'), 
#              os.path.join('data/labels/valid', str(l2)+'.txt'))



# for label in labels:
#     print(label)
#     print(label.split('.')[:-1])
#     break


In [10]:
for image_name in tqdm(images[:int(0.2*len(images))]):
    l1 = image_name
    l2 = '.'.join(l1.split('.')[:-1])

#     move(os.path.join('data/images/train', l1), 
#              os.path.join('data/images/valid', l1))
    if os.path.exists(os.path.join('data/labels/train', str(l2)+'.txt')):
            move(os.path.join('data/labels/train', str(l2)+'.txt'), 
                 os.path.join('data/labels/valid', str(l2)+'.txt'))
            move(os.path.join('data/images/train', l1), 
             os.path.join('data/images/valid', l1))

#     move(os.path.join(images, image_name), 
#          os.path.join('data/images/valid', image_name))
#     l1 = image_name.split('.')[:-1]
#     l1 = '.'.join(l1)
#     if os.path.exists(os.path.join(labels, str(l1)+'.txt')):
#         move(os.path.join(labels, str(l1)+'.txt'), 
#              os.path.join('data/labels/valid', str(l1)+'.txt'))

  0%|          | 0/339 [00:00<?, ?it/s]

In [7]:
save_dir = 'LADD'
os.makedirs(save_dir, exist_ok=True)
dirs = ['LDD/summer_moscow_2019', 'LDD/spring_korolev_2019', 'LDD/summer_tambov_2019', 'LDD/winter_moscow_2018']

In [8]:
for directory in dirs:
    copytree(directory, save_dir, dirs_exist_ok=True)

### convert to yolo format

In [9]:
!pip install xmltodict

In [10]:
import xmltodict
def xml2yolo(filepath, save_path):
    with open(filepath, 'r') as f:
        data = f.readlines()
    annot = xmltodict.parse(' '.join(data))
    img_h = int(annot['annotation']['size']['height'])
    img_w = int(annot['annotation']['size']['width'])
    peoples = []
    if 'object' not in annot['annotation'].keys():
        return
    if isinstance(annot['annotation']['object'], dict):
        annot['annotation']['object'] = [annot['annotation']['object']]
    for obj in annot['annotation']['object']:
        bbox = obj['bndbox']
        y_min, x_min, y_max, x_max = int(bbox['ymin']),  int(bbox['xmin']),  int(bbox['ymax']),  int(bbox['xmax'])
        w, h = x_max - x_min, y_max - y_min
        x_c = x_min + w//2
        y_c = y_min + h//2
        w /= img_w
        h /= img_h
        x_c /= img_w
        y_c /= img_h
        peoples.append(f"0 {x_c} {y_c} {w} {h}")
    
    write_txt(peoples, save_path)

def write_txt(data, save_path):
    with open(save_path, 'w') as f:
        for line in data:
            f.write(f"{line}\n")

In [11]:
labels_dir = 'LADD/labels'
annot_dir = 'LADD/Annotations'
os.makedirs(labels_dir, exist_ok=True)

In [12]:
for file in tqdm(os.listdir(annot_dir)):
    xml2yolo(os.path.join(annot_dir, file), os.path.join(labels_dir, file.split('.')[0]+'.txt'))

  0%|          | 0/1422 [00:00<?, ?it/s]

### TRAIN SET

In [ ]:
import pandas as pd
import json
import cv2

def parse_shape(shape):
    data = []
    start = []
    end = []
    
    for index, s in enumerate(shape):
        if s=='{':
            start.append(index)
        if s=='}':
            end.append(index+1)
    
    for s, e in zip(start, end):
        data.append(json.loads(shape[s:e]))
    return data

def shape2yolo(shape, img_path, save_path):
    data = parse_shape(shape)
    h, w, _ = cv2.imread(img_path).shape
    peoples = []
    
    for i in data:
        x_c, y_c, w_2 = i['cx'], i['cy'], i['r']
        w_2 = w_2*2/w
        x_c /= w
        y_c /= h
        peoples.append(f"0 {x_c} {y_c} {w_2} {w_2}")
    
    write_txt(peoples, save_path)

In [ ]:
df = pd.read_csv('train.csv', delimiter=',')
df[df['count_region']>0].head()

In [ ]:
train_path_basedataset = 'train'
omsk_dataset_full = 'omsk_full'
LADD_images = 'LADD/JPEGImages'
LADD_labels = 'LADD/labels'

os.makedirs(os.path.join(omsk_dataset_full, 'images'), exist_ok=True)
os.makedirs(os.path.join(omsk_dataset_full, 'labels'), exist_ok=True)
os.makedirs(os.path.join(omsk_dataset_full, 'test/images'), exist_ok=True)
os.makedirs(os.path.join(omsk_dataset_full, 'test/labels'), exist_ok=True)
os.makedirs(os.path.join(omsk_dataset_full, 'train/images'), exist_ok=True)
os.makedirs(os.path.join(omsk_dataset_full, 'train/labels'), exist_ok=True)

for index, row in df[df['count_region']>0].iterrows():
    copyfile(os.path.join(train_path_basedataset, row['ID_img']), 
             os.path.join(omsk_dataset_full, 'train/images', row['ID_img']))
    shape = row['region_shape']
    shape2yolo(shape, os.path.join(train_path_basedataset, row['ID_img']), 
               os.path.join(omsk_dataset_full, 'train/labels', row['ID_img'].split('.')[0]+'.txt'))

In [ ]:
copytree('LADD/labels', os.path.join(omsk_dataset_full, 'labels'), dirs_exist_ok=True)
copytree('LADD/JPEGImages', os.path.join(omsk_dataset_full, 'images'), dirs_exist_ok=True)

In [ ]:
images = os.listdir(LADD_images)
random.shuffle(images)

for image_name in images[:int(0.2*len(images))]:
    move(os.path.join(LADD_images, image_name), 
         os.path.join(omsk_dataset_full, 'test/images', image_name))
    if os.path.exists(os.path.join(LADD_labels, image_name.split('.')[0]+'.txt')):
        move(os.path.join(LADD_labels, image_name.split('.')[0]+'.txt'), 
             os.path.join(omsk_dataset_full, 'test/labels', image_name.split('.')[0]+'.txt'))
        
move(LADD_images, os.path.join(omsk_dataset_full, 'train'))
move(LADD_labels, os.path.join(omsk_dataset_full, 'train'))

# added empty images

In [ ]:
empty_images = df[df['count_region']==0]['ID_img'].to_list()
random.shuffle(empty_images)

for image in empty_images[:350]:
    copyfile(os.path.join(train_path_basedataset, image), 
             os.path.join(omsk_dataset_full, 'train/images', image))
    
for image in empty_images[-350:]:
    copyfile(os.path.join(train_path_basedataset, image), 
             os.path.join(omsk_dataset_full, 'test/images', image))

In [ ]:
# make train and test

# TRAIN

In [ ]:
!git clone https://github.com/ultralytics/yolov5
cd "yolov5"
!pip install -r requirements.txt

In [ ]:
!python train.py --img 1280 --batch -1 --epochs 100 --data "/home/jovyan/omsk_hack.yaml" --weights yolov5m6.pt --project "hackaton_omsk_find_people" --name "yolov5m6"

In [ ]:
!python detect.py --source "/home/jovyan/test" --weights "/home/jovyan/yolov5/hackaton_omsk_find_people/yolov5m6/weights/best.pt" --save-txt --save-conf --name "yolov5m6_people_test" --imgsz 1280 --exist-ok --conf-thres 0.2

In [ ]:
def read_txt(label_path, img_path, th=0.3):
    with open(os.path.join(label_path), 'r') as file:
        lines = file.readlines()
    h, w, _ = cv2.imread(img_path).shape
    lines = [line.rstrip().split(' ') for line in lines]
    result = []
    for line in lines:
        cl, xc, yc, w_, h_, t = list(map(float, line))
        xc*=w
        yc*=h
        w_*=w
        h_*=h
        if t>=th:
            result.append(f"{{\"cx\":{xc},\"cy\":{yc},\"r\":{max(w_,h_)}}}")
    return result

In [ ]:
labels = '/home/jovyan/yolov5/runs/detect/yolov5m6_people_test/labels'
test_images_path = '/home/jovyan/hack/find_people/test'
res = {}
for file in tqdm(os.listdir(labels)):
    if os.path.exists(os.path.join(test_images_path, file.split('.')[0]+'.JPG')):
        predict =read_txt(os.path.join(labels, file), os.path.join(test_images_path, file.split('.')[0]+'.JPG'))
    else: 
        predict=read_txt(os.path.join(labels, file), os.path.join(test_images_path, file.split('.')[0]+'.jpg'))
    res[file.split('.')[0]] = predict

In [ ]:
df = pd.DataFrame(columns=['ID_img','region_shape'])

for file in tqdm(os.listdir(test_images_path)):
    if file.split('.')[0] not in res:
        res[file.split('.')[0]] = 0
    df = df.append({'ID_img':file,'region_shape':res[file.split('.')[0]]}, ignore_index=True)
    
df.to_csv(r'/home/jovyan/omsk_solution.csv', index=False)